# Processing a (fake) globular cluster with the DM Science Pipelines

**Last verified to run:** 2019-10-15

**Verified science platform release or release candidate:** 18.1.0

This notebook provides an introduction to using some of the most important pieces of the DM Science Pipelines codebase:

 - Geometry classes from `lsst.geom`, such as points and boxes.
 - Higher-level astronomical primitives from `lsst.afw`, such as the `Image`, `Exposure`, and `Psf` classes.
 - Our core algorithmic `Task` classes, including those for source detection, deblending, and measurement.
 
We'll be working with coadded images made from Subaru Hyper Suprime-Cam (HSC) data in the COSMOS field, augmented with a simulated globular cluster.  We've taken a recent LSST reprocessing of the HSC-SSP UltraDeep COSMOS field (see [this page](https://confluence.lsstcorp.org/display/DM/S18+HSC+PDR1+reprocessing) for information on that reprocessing, and [this page](https://hsc-release.mtk.nao.ac.jp/doc/) for the data), and added simulated stars from a scaled [SDSS catalog](http://www.sdss.org/dr14/data_access/value-added-catalogs/?vac_id=photometry-of-crowded-fields-in-sdss-for-galactic-globular-and-open-clusters).  The result is a very deep image (deeper than the 10-year LSST Deep-Wide-Fast survey, though not as deep as LSST Deep Drilling fields will be) with both a large number of galaxies and region full of stars.  As we'll see, that'll present a challenge for the vanilla DM pipelines (at least today), and hence a good excuse to do some custom processing.

## Imports and Custom Installs

We'll start with some standard imports of both LSST and third-party packages.

These are automatically installed in the LSST Science Platform Notebook Aspect.

In [ ]:
import numpy as np
from lsst.daf.persistence import Butler
from lsst.geom import Box2I, Box2D, Point2I, Point2D, Extent2I, Extent2D
from lsst.afw.image import Exposure, Image, PARENT

## Reading Data

We'll be retrieving data using the `Butler` tool, which manages where various datasets are stored on the filesystem (and can in principle manage datasets that aren't even stored as files, though all of these are).

We start by creating a `Butler` instance, pointing it at a *Data Repository* (which here is just a root directory).  If you're interesting in looked at the original HSC data without the simulated cluster, change the path below to just `/datasets/hsc/cosmos`.

In [ ]:
butler = Butler("/project/jbosch/tutorials/lsst2018/data")

Datasets managed by a butler are identified by a dictionary *Data ID* (specifying things like the visit number or sky patch) and a string *DatasetType* (such as a particular image or catalog).  Different DatasetTypes have different keys, while different instances of the same Dataset Type have different values.  All of the datasets we use in this tutorial will correspond to the same patch of sky, so they'll have at least the keys in the dictionary in the next cell (they will also have `filter`, but with different values):

In [ ]:
dataId = {"tract": 9813, "patch": "4,4"}

We can now use those to load a set of *griz* coadds, which we'll put directly in a dictionary.  The result of each `Butler.get` call is in this case an `lsst.afw.image.Exposure` object, an image that actually contains three "planes" (the main image, a bit mask, and a variance image) as well as many other objects that describe the image, such as its PSF and WCS.  Note that we (confusingly) use `Exposures` to hold coadd images as well as true single-exposure images.

The DatasetType here is `deepCoadd_calexp` (a coadd on which we've already done some additional processing, such as subtracting the background and setting some mask values), and the extra `filter` argument gets appended to the Data ID.

In [ ]:
coadds = {b: butler.get("deepCoadd_calexp", dataId, filter="HSC-{}".format(b.upper())) for b in "griz"}

## Making and displaying color composite images

We'll start by just looking at the images, as 3-color composites.  We'll use astropy to build those as a nice way to demonstrate how to get NumPy arrays from the `Exposure` objects in the `coadds` dict.  (LSST also has code to make 3-color composites using the same algorithm, and in fact the Astropy implementation is based on ours, but now that it's in Astropy we'll probably retire ours.)

We'll just use matplotlib to display the images themselves.  We'll use Firefly for other image display tasks later, but while Firefly itself supports color-composites, we haven't finished connecting that functionality to the Python client we'll demonstrate here.

In [ ]:
from astropy.visualization import make_lupton_rgb
from matplotlib import pyplot
%matplotlib inline

We'll use the following function a few times to display color images.  It's worth reading through the implementation carefully to see what's going on.

In [ ]:
def showRGB(exps, bgr="gri"):
    """Display an RGB color composite image with matplotlib.
    
    Parameters
    ----------
    exps : `dict`
        Dictionary of `lsst.afw.image.Exposure` objects, keyed by filter name.
    bgr : sequence
        A 3-element sequence of filter names (i.e. keys of the exps dict) indicating what band
        to use for each channel.
    """
    # Extract the primary image component of each Exposure with the .image property, and use .array to get a NumPy array view.
    rgb = make_lupton_rgb(image_r=exps[bgr[2]].image.array,  # numpy array for the r channel
                          image_g=exps[bgr[1]].image.array,  # numpy array for the g channel
                          image_b=exps[bgr[0]].image.array,  # numpy array for the b channel
                          stretch=1, Q=10)  # parameters used to stretch and scale the pixel values
    pyplot.figure(figsize=(20, 15))
    # Exposure.getBBox() returns a Box2I, a box with integer pixel coordinates that correspond to the centers of pixels.
    # Matplotlib's `extent` argument expects to receive the coordinates of the edges of pixels, which is what
    # this Box2D (a box with floating-point coordinates) represents.
    integerPixelBBox = exps[bgr[0]].getBBox()
    bbox = Box2D(integerPixelBBox)
    pyplot.imshow(rgb, interpolation='nearest', origin='lower', extent=(bbox.getMinX(), bbox.getMaxX(), bbox.getMinY(), bbox.getMaxY()))

In [ ]:
showRGB(coadds, bgr="gri")

In [ ]:
showRGB(coadds, bgr="riz")

Those images are a full "patch", which is our usual unit of processing for coadds - it's about the same size as a single LSST sensor (exactly the same in pixels, smaller in terms of area because these use HSC's smaller pixel scale).  That's a bit unweildy (just because waiting for processing to happen isn't fun in a tutorial setting), so we'll reload our dict with sub-images centered on the cluster.  Note that we can load the sub-images directly with the `butler`, by appending `_sub` to the DatasetType and passing a `bbox` argument.

In [ ]:
coadds = {b: butler.get("deepCoadd_calexp_sub", dataId, filter="HSC-{}".format(b.upper()),
                        bbox=Box2I(corner=Point2I(18325, 17725), dimensions=Extent2I(400, 350)))
          for b in "griz"}

In [ ]:
showRGB(coadds, "gri")

## Basic Processing

Now we'll try the regular LSST processing tasks, with a simpler configuration than we usually use to process coadds, just to avoid being distracted by complexity.  This includes

 - Detection (`SourceDetectionTask`): given an `Exposure`, find above-threshold regions and peaks within them (`Footprints`), and create a *parent* source for each `Footprint`.
 - Deblending (`SourceDeblendTask`): given an `Exposure` and a catalog of parent sources, create a *child* source for each peak in every `Footprint` that contains more than one peak.  Each child source is given a `HeavyFootprint`, which contains both the pixel region that source covers and the fractional pixel values associated with that source.
 - Measurment (`SingleFrameMeasurementTask`): given an `Exposure` and a catalog of sources, run a set of "measurement plugins" on each source, using deblended pixel values if it is a child.

We'll start by importing these, along with the `SourceCatalog` class we'll use to hold the outputs.

In [ ]:
from lsst.meas.algorithms import SourceDetectionTask
from lsst.meas.deblender import SourceDeblendTask
from lsst.meas.base import SingleFrameMeasurementTask
from lsst.afw.table import SourceCatalog

We'll now construct all of these `Tasks` before actually running any of them.  That's because `SourceDeblendTask` and `SingleFrameMeasurementTask` are constructed with a `Schema` object that records what fields they'll produce, and they modify that schema when they're constructed by adding columns to it.  When we run the tasks later, they'll need to be given a catalog that includes all of those columns, but we can't add columns to a catalog that already exists.

To recap, the sequence looks like this:

 1. Make a (mostly) empty schema.
 2. Construct all of the `Task`s (in the order you plan to run them), which adds columns to the schema.
 3. Make a `SourceCatalog` object from the *complete* schema.
 4. Pass the same `SourceCatalog` object to each `Task` when you run it.

In [ ]:
schema = SourceCatalog.Table.makeMinimalSchema()

detectionTask = SourceDetectionTask(schema=schema)

deblendTask = SourceDeblendTask(schema=schema)

# We'll customize the configuration of measurement to just run a few plugins.
# The default list of plugins is much longer (and hence slower).
measureConfig = SingleFrameMeasurementTask.ConfigClass()
measureConfig.plugins.names = ["base_SdssCentroid", "base_PsfFlux", "base_SkyCoord"]
# "Slots" are aliases that provide easy access to certain plugins.
# Because we're not running the plugin these slots refer to by default,
# we need to disable them in the configuration.
measureConfig.slots.apFlux = None
measureConfig.slots.gaussianFlux = None
measureConfig.slots.shape = None
measureConfig.slots.modelFlux = None
measureConfig.slots.calibFlux = None
measureTask = SingleFrameMeasurementTask(config=measureConfig, schema=schema)

In [ ]:
list(measureConfig.slots.keys())

The first step we'll run is detection, which actually returns a new `SourceCatalog` object rather than working on an existing one.

Instead, it takes a `Table` object, which is sort of like a factory for records.  We won't use it directly after this, and it isn't actually necessary to make a new `Table` every time you run `SourceDetectionTask` (but you can only create one after you're done adding columns so the schema).

`Task`s that return anything do so via a `lsst.pipe.base.Struct` object, which is just a simple collection of named attributes.  The only return values we're  interested is `sources`.  That's our new `SourceCatalog`.

In [ ]:
table = SourceCatalog.Table.make(schema)
detectionResult = detectionTask.run(table, coadds['r'])
catalog = detectionResult.sources

Let's take a quick look at what's in that catalog.  First off, we can look at its schema:

In [ ]:
catalog.schema

Note that this includes a lot of columns that were actually added by the deblend or measurement steps; those will all still be blank (`0` for integers or flags, `NaN` for floating-point columns).

In fact, the only columns filled by `SourceDetectionTask` are the IDs.  But it also attaches `Footprint` objects, which don't appear in the schema.  You can retrieve the `Footprint` by calling `getFootprint()` on a row:

In [ ]:
footprint = catalog[0].getFootprint()

`Footprints` have two components:
 - a `SpanSet`, which represents an irregular region on an image via a list of (y, x0, x1) `Spans`;
 - a `PeakCatalog`, a slightly different kind of catalog whose rows represent peaks within that `Footprint`.

In [ ]:
print(footprint.getSpans())

In [ ]:
print(footprint.getPeaks())

It's worth noting that while the peaks *can* have both an integer-valued position and a floating-point position, they're the same right now; `SourceDetectionTask` currently just finds the pixels that are local minima and doesn't try to find their sub-pixel locations.  That's left to the centroider, which is part of the measurement stage.

Before we can get to that point, we need to run the deblender:

In [ ]:
deblendTask.run(coadds['r'], catalog)

`SourceDeblendTask` doesn't actually return anything - all of its outputs are just modifications to the catalog that's passed in.  It both sets some columns (`parent` and all those whose names start with `deblend_`) and creates new rows (for the child sources).  It does *not* remove the parent rows it created those child rows from, and this is intentional, because we want to measure both "interpretations" of the blend family: one in which there is only one object (the parent version) and one in which there are several (the children). Before doing any science with the outputs of an LSST catalog, it's important to remove one of those interpretations (typically the parent one).  That can be done by looking at the `deblend_nChild` and `parent` fields:

 - `parent` is the ID of the source from which this was deblended, or `0` if the source is itself a parent.
 - `deblend_nChild` is the number of child sources this source has (so it's `0` for sources that are themselves children or were never blended).
 
Together, these define two particularly useful filters:

 - `deblend_nChild == 0`: never-blended object or de-blended child
 - `deblend_nChild == 0 and parent == 0`: never-blended object
 
The first is what you'll usually want to use; the second is what to use if you're willing to throw away some objects (possibly many) because you don't trust the deblender.

The last processing step for our purposes is running measurement:

In [ ]:
measureTask.run(catalog, coadds['r'])

## Inspecting processing outputs with Firefly

Now we'll look at the results using Firefly.  Because these are the low-level outputs of a particular configuration of some processing `Tasks`, Firefly doesn't know nearly as much about them as it will about the (more "curated") data products that will appear in actual LSST Data Releases.  The tooling to connect Firefly to Python is also pretty new, so you can expect a lot of improvement in the future in both what Firefly can do with the datasets we send it and how easy it is is to send them.

First some imports and setup:

In [ ]:
from lsst.afw.display import setDefaultBackend, Display
setDefaultBackend("firefly")

In LSST Science Platform environments, we have prepopulated environment variables `FIREFLY_URL` and `FIREFLY_HTML` with the Firefly server URL and the landing page or html file on that server. These are picked up automatically when creating the display.

In [ ]:
display = Display(frame=1)

In the LSST Science Platform, defining the Display opens a browser tab. A nice way to arrange your Jupyterlab windows is to put the notebooks on the right-hand side, and the Firefly tabs on the left half.

We can now display an `Exposure` object:

In [ ]:
display.mtv(coadds["r"])

After doing that, take some time to play around with the GUI. See [this help page](http://irsa.ipac.caltech.edu/onlinehelp/finderchart/visualization.html?bd=2018-07-12#imageoptions) for help with the Firefly toolbar. 

There are a lot of transparent mask overlays that can make it hard to see the image, but the overlays button ![overlays button](http://irsa.ipac.caltech.edu/onlinehelp/finderchart/img/layers.png) on the toolbar gives you very detailed control over them. Mask transparency can also be controlled using `afw.display`, for individual planes or for all.

In [ ]:
display.setMaskTransparency(80)
display.setMaskTransparency(90, 'DETECTED')

These transparency settings are "sticky" for each Display, and settable by default with `lsst.afw.display.setDefaultMaskTransparency`. Alternatively, we can set the transparency after each image display.

The scale can also be changed programmatically, or by `afw.display` commands.

In [ ]:
display.scale("asinh", "zscale")

The function below overplots the sources we detected on the image.  Firefly itself has much more sophisticated ways of interacting them with catalogs; an example of overlaying a catalog will be shown at the end of the notebook.

In [ ]:
def showCatalog(disp, cat, color='orange', sym='+'):
    """Display sources from a catalog.
    
    Parameters
    ----------
    disp : `lsst.afw.display.Display`
        Display interface object to send to.
    cat : `lsst.afw.table.SourceCatalog`
        Catalog containing sources to display.  Must have populated centroid columns.
    color : `str`
        Color to use for overlaid points (X11 color string).
    sym : `str`
        Symbol; one of "+", "x", or "o".
    """
    with disp.Buffering():
        for record in cat:
            if record["deblend_nChild"] != 0:  # don't show "one object" interpretation of blends
                continue
            disp.dot(sym, record.getX(), record.getY(), size=5, ctype=color)

In [ ]:
showCatalog(display, catalog)

Specific to the `lsst.display.firefly` backend is facility for overlaying LSST source detection footprints. An example is [shown in the module documentation](https://pipelines.lsst.io/modules/lsst.display.firefly/viewing-footprints.html).

In [ ]:
display.overlayFootprints(catalog,
                          color='rgba(74,144,226,0.50)',
                          highlightColor='yellow', selectColor='orange',
                          style='outline', layerString='detection footprints ',
                          titleString='catalog footprints ')

A table viewer for the catalog appears in the Firefly tab. It is possible to filter the `category` column to show only 'deblended child' footprints.

After experimenting with footprint viewer, you can close the 'catalog footprints 1' tab to carry on with the rest of this tutorial.

# Subtracting Stars

To see how well that measurement worked, we'll use the PSF model and the measured PSF fluxes to subtract all of the objects.  Some of those objects are actually well-resolved galaxies, so we don't expect them to subtract well, but we'll just ignore that for now and focus our attention on the stars when we look at the results.

Once again, it's worth taking some time to read carefully through the function below before just running it.

In [ ]:
def subtractStars(data, cat, fluxes=None):
    """Subtract point sources from an image.
    
    Parameters
    ----------
    data : `lsst.afw.image.Exposure`
        Image from which point sources will be subtracted (not modified).
    cat : `lsst.afw.table.SourceCatalog`
        Catalog providing centroids and possibly fluxes.
    fluxes : `numpy.ndarray`, optional
        If not None, an array of the same length as ``cat`` containing fluxes
        to use.  If None, ``cat.getPsfFlux()` will be used instead.
        
    Returns
    -------
    model : `lsst.afw.image.Exposure`
        Image containing just the point source models.
    residuals : `lsst.afw.image.Exposure`
        ``data`` with ``model`` subtracted.
    """
    # Make sure no one calls this still-blended sources in the catalog.
    assert (cat["deblend_nChild"] == 0).all()
    # Get the PSF model from the given Exposure object.  The returned object
    # can be evaluated anywhere in the image to obtain an image of the PSF at
    # that point.
    psf = data.getPsf()
    # Make a new blank Exposure object with the same dimensions and pixel type.
    model = Exposure(data.getBBox(), dtype=np.float32)
    # Copy the WCS and PSF over from the original Exposure.
    model.setWcs(data.getWcs())
    model.setPsf(psf)
    if fluxes is None:
        fluxes = cat.getPsfInstFlux()
    for flux, record in zip(fluxes, cat):
        # Obtain a PSF model image at the position of this source
        psfImage = psf.computeImage(record.getCentroid())
        # Make sure the PSF model image fits within the larger image; if it doesn't, clip it so it does.
        psfBBox = psfImage.getBBox()
        if not data.getBBox().contains(psfBBox):
            psfBBox.clip(data.getBBox())  # shrink the bounding box to the intersection
            psfImage = psfImage[psfBBox, PARENT]   # obtain a subimage
        # Make a subimage view of `model`, and subtract the PSF image, scaled by the flux.
        # PARENT here sets the coordinate system to be the one shared by all patches in
        # the tract rather than the one in which this patches' origin is (0, 0).
        # PARENT is the coordinate system used by the PSF, and it will soon be the default
        # here too (but isn't yet, so we need to make that explicit).
        model.image[psfBBox, PARENT].scaledPlus(flux, psfImage.convertF())
    # Now that we've made a model image, make a copy of the data and subtract the model from it.
    residuals = data.clone()
    residuals.image -= model.image
    return model, residuals

Before we can actually subtract the stars, we should remove the not-deblended sources; this function and the next one we write will assume they've been removed.

We can do that with NumPy-style boolean indexing, with one catch:

In [ ]:
deblended = catalog[catalog["deblend_nChild"] == 0].copy(deep=True)

Column arrays of `SourceCatalogs` can only be accessed when the catalog is stored in a single contiguous block of memory.  But unlike Numpy arrays, using boolean indexing on a catalog doesn't automatically make a copy to ensure memory is contiguous.  Instead it creates a view to the selected rows.  That can be useful or more efficient in some cases, but it also prevents us from accessing columns.  To fix that, we immediately make a deep copy of the catalog, which copies it into a new block of contiguous memory.

We can now run our `subtractStars` function.

In [ ]:
model, residuals = subtractStars(coadds['r'], deblended)

To look at the results, we'll make two more display frames to show the residuals (frame 2) and model (frame 3).

Use the "WCS Match" and single-frame view GUI options to blink between them and the original image.

In [ ]:
display2 = Display(frame=2)#, name=channel)
display2.setMaskTransparency(80)
display2.setMaskTransparency(90, 'DETECTED')
display2.mtv(residuals, title="residuals")

In [ ]:
display3 = Display(frame=3)#, name=channel)
display3.mtv(model, title="model")

The isolated objects seem to be subtracted well, and *some* of the deblended ones are too.  But others really didn't subtract well at all; the deblender is probably mangling those.  The current deblender depends a lot on having at least some sides of an object not blended with a neighbor, and that's manifestly untrue in the center of the cluster.  We're currently working on a new deblender ([Scarlet](https://github.com/fred3m/scarlet)) that we expect to handle this better.

In any case, if we look at the overplotted positions, it looks like the centroids aren't too bad, even if the fluxes frequently are.  It seems our centroider is pretty robust to whatever failure modes the deblender is exhibiting.

# Fitting Stars

For better photometry, let's try fitting the stars ourselves, simultaneously.  We can use the same PSF object we used to subtract stars to instead construct a model for the entire image, and then we can use basic linear least squares from NumPy to fit it.

Once again, this will go poorly for galaxies, and we'll ignore that.  We could also imagine adding terms to fit small offsets in the positions, or using sparse matrices to make this scale better.  But this works well enough for this tutorial.

This is another function worth reading through carefully.

In [ ]:
def fitStars(data, cat):
    """
    Fit all sources in the given catalog simultaneously, assuming they are all point sources.
    
    Parameters
    ----------
    data : `lsst.afw.image.Exposure`
        Image to fit and subtract model from.
    cat : `lsst.afw.table.SourceCatalog`.
        Catalog providing centroids.
    
    Returns
    -------
    fluxes : `numpy.ndarray`
        Array of best-fit flux values.
    model : `lsst.afw.image.Exposure`
        Image containing just the point source models.
    residuals : `lsst.afw.image.Exposure`
        ``data`` with ``model`` subtracted.
    """
    assert (cat["deblend_nChild"] == 0).all()
    bbox = data.getBBox()
    psf = data.getPsf()
    # Dimensions of the problem:
    N = len(cat)
    W = bbox.getWidth()
    H = bbox.getHeight()
    M = W*H
    # To solve a lineat-least squares problem, we need to construct a matrix that maps
    # parameters (fluxes) to data (pixel values).  That'd have dimensions M x N.
    # Instead, we'll construct an array with dimensions N x H x W, which we'll later
    # transpose and reshape.  It's important that we order the dimensions this way
    # because it makes each nested H x W array have the same memory layout as the
    # LSST Image class, which lets us make image views into those subarrays.
    matrix = np.zeros((N, H, W), dtype=float)
    for n, record in enumerate(cat):
        # Make an Image view to a nested sub-array.  Note that writing to this
        # will modify the parent array.
        matrixView = Image(matrix[n, :, :], xy0=bbox.getMin(), dtype=np.float64)
        # Obtain a PSF image, and clip it to fit the larger image, just as we
        # did in subtractStars().
        psfImage = psf.computeImage(record.getCentroid())
        psfBBox = psfImage.getBBox()
        if not bbox.contains(psfBBox):
            psfBBox.clip(bbox)
            psfImage = psfImage[psfBBox, PARENT]
        # Add the PSF image to the matrix sub-array.
        matrixView[psfBBox, PARENT].scaledPlus(record.getPsfInstFlux(), psfImage)
    # Reshape and transpose the matrix, as promised
    A = matrix.reshape(N, M).transpose()
    # Get an array view to the image we're fitting, and flatten it the same way.
    b = data.image.array.reshape(M)
    # Fit for the fluxes.  This does an SVD under the hood.
    fluxes, _, _, _ = np.linalg.lstsq(A, b, rcond=None)
    # Make a model image and subtract it from the data.  Because we've already
    # evaluated the matrix, this is much more efficient than calling
    # subtractStars with our flux array.
    model = Exposure(bbox, dtype=np.float32)
    model.setWcs(data.getWcs())
    model.setPsf(psf)
    model.image.array[:, :] = np.dot(A, fluxes).reshape(bbox.getHeight(), bbox.getWidth())
    residuals = data.clone()
    residuals.image -= model.image
    return fluxes, model, residuals

In [ ]:
fluxes, model, residuals = fitStars(coadds['r'], deblended)

Display the new residuals (omitting the masks):

In [ ]:
display2.mtv(residuals.image, title="residuals")
display3.mtv(model, title="model")

The results are better than the last subtraction, but still not great.

Note that you may need to set the stretch manually (color stretch icon ![stretch button](http://irsa.ipac.caltech.edu/onlinehelp/finderchart/img/stretch.png)) to give the residuals image the same stretch as the others.  For this image, you can set the stretch using "Data" bounds of `(-0.05, 2)` with the original image selected, and then blinking to the other frames while the stretch window is open. The lock icon ![lock icon](http://irsa.ipac.caltech.edu/onlinehelp/finderchart/img/lockimages.png) on the Firefly toolbar can be used to lock the stretch for the displayed frames.

The stretch is also settable programmatically:

In [ ]:
display.scale('asinh', -0.1, 0.5, Q=8)
display2.scale('asinh', -0.1, 0.5, Q=8)
display3.scale('asinh', -0.1, 0.5, Q=8)

We can erase the overlays on the first display:

In [ ]:
display.erase()

## Iterating

To improve things further, let's try running the detect/deblend/measure `Tasks` on the *residual* image, to pick up those peaks that just weren't included in the first round.

Here's a function that just runs those (relying on us having constructed them all earlier; we don't have to re-construct them to run them again):

In [ ]:
def process(data):
    """Run detection, deblending, and measurement, returning a new SourceCatalog.
    """
    result = detectionTask.run(table, data)
    cat = result.sources
    deblendTask.run(data, cat)
    measureTask.run(cat, data)
    return cat[cat["deblend_nChild"] == 0].copy(deep=True)

In [ ]:
additional = process(residuals)

To make use of those additional sources, we'll have to merge them with the original ones.  We'll do that extremely naively - we won't even worry about whether we have any duplicates (from sources that were partially - but not completely - subtracted by the model).

Here's a function to do that merge.  Note that we reserve space in the output catalog before actually concatenating the input catalogs into it.  That makes sure the result is contiguous in memory and hence we can get column arrays without doing a deep copy at the end.

In [ ]:
def concatenate(*cats):
    """Concatenate multiple SourceCatalogs (assumed to have the same schema)."""
    result = SourceCatalog(table)
    result.reserve(sum(len(cat) for cat in cats))
    for cat in cats:
        result.extend(cat)
    return result

In [ ]:
combined = concatenate(deblended, additional)

Now we'll fit all the stars in the combine catalog together, and look at the residuals again:

In [ ]:
fluxes, model, residuals = fitStars(coadds['r'], combined)

In [ ]:
display2.mtv(residuals.image, title="residuals")
display3.mtv(model, title="model")

Even better!  One more iteration *might* make things better, but we're probably getting to the limits of this very simple algorithm (especially since there really are a lot of galaxies in this image).

## Uploading the catalog directly to Firefly

As noted earlier, Firefly has sophisticated capabilities for viewing tables and overlaying catalogs. The `firefly_client` Python package underlies the Firefly backend in `lsst.afw.display`. The `firefly_client.plot` module includes convenience functions that we'll demonstrate here.

In [ ]:
import firefly_client.plot as ffplt

The `firefly_client.plot` module should default to the client that is used in the `afw.display` Displays we defined earlier, so long as a browser tab is connected to those displays. To be certain, the next line ensures we use the same setup as our Display instances.

In [ ]:
ffplt.use_client(display.getClient())

Upload our combined catalog to Firefly. By default, the catalog will be shown in an interactive table viewer. Since the catalog contains coordinate columns recognized by Firefly, the catalog entries will be overlaid on the images.

In [ ]:
tbl_id = ffplt.upload_table(combined, title='combined catalog')

Firefly includes plotting capabilities using the Plotly.js library. We'll end this section with a histogram of flux values. `ffplt.scatter` can be used to make a scatter plot of two columns for a table. These functions use the table ID of the last uploaded table by default; the `tbl_id` we saved in the last cell can be passed as an optional argument.

In [ ]:
ffplt.hist('log10(base_PsfFlux_instFlux)')

The table, plot, and image cells can be resized and moved around in the Firefly viewer.